In [1]:
from seq2seq.gSCAN_dataset import GroundedScanDataset
from seq2seq.model import Model
from seq2seq.rollout import Rollout
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
import inspect
import numpy as np
device = torch.device(type='cuda')
%load_ext autoreload
%autoreload 2

In [2]:
training_set = GroundedScanDataset("gSCAN_data/data/compositional_splits/dataset.txt", "gSCAN_data/data/compositional_splits/", split="train", 
                                        input_vocabulary_file="training_input_vocab.txt", 
                                        target_vocabulary_file="training_target_vocab.txt", 
                                        generate_vocabulary=False, k=0) 

In [3]:
testing_set = GroundedScanDataset("gSCAN_data/data/compositional_splits/dataset.txt", "gSCAN_data/data/compositional_splits/", split="test", 
                                        input_vocabulary_file="training_input_vocab.txt", 
                                        target_vocabulary_file="training_target_vocab.txt", 
                                        generate_vocabulary=False, k=0) 

In [18]:
example = training_set.dataset.get_examples_with_image("train", True)

In [ ]:
for i, e in enumerate(example):
    print(e)
    input_commands = e["input_command"]
    target_commands = e["target_command"]
    situation_image = e["situation_image"]
    plt.imshow(situation_image[:,:,3])
    if i == 2:
        break

In [21]:
test_example = testing_set.dataset.get_examples_with_image("test", True)

In [ ]:
for i, e in enumerate(test_example):
    print(e)
    input_commands = e["input_command"]
    target_commands = e["target_command"]
    situation_image = e["situation_image"]
    plt.imshow(situation_image[:,:,3])
    break

In [4]:
training_set.read_dataset(max_examples=100,
                              simple_situation_representation=True)

In [5]:
cfg = {'data_path': 'gSCAN_data/data/compositional_splits/dataset.txt', 'data_directory': 'gSCAN_data/data/compositional_splits/', 'generate_vocabularies': False, 'input_vocab_path': 'training_input_vocab.txt', 'target_vocab_path': 'training_target_vocab.txt', 'embedding_dimension': 25, 'num_encoder_layers': 1, 'encoder_dropout_p': 0.3, 'encoder_bidirectional': True, 'training_batch_size': 50, 'test_batch_size': 1, 'max_decoding_steps': 30, 'num_decoder_layers': 1, 'decoder_dropout_p': 0.3, 'cnn_kernel_size': 7, 'cnn_dropout_p': 0.1, 'cnn_hidden_num_channels': 50, 'simple_situation_representation': True, 'decoder_hidden_size': 100, 'encoder_hidden_size': 100, 'learning_rate': 0.001, 'adam_beta_1': 0.9, 'adam_beta_2': 0.999, 'resume_from_file': '', 'max_training_iterations': 200000, 'output_directory': 'models', 'print_every': 100, 'evaluate_every': 1000, 'conditional_attention': True, 'auxiliary_task': False, 'weight_target_loss': 0.3, 'attention_type': 'bahdanau', 'k': 0, 'max_training_examples': None, 'seed': 42, 'kwargs': {'mode': 'train', 'split': 'test', 'max_testing_examples': None, 'splits': 'test', 'output_file_name': 'predict.json'}, 'device': device, 'lr_decay': 0.9, 'lr_decay_steps': 20000}

In [6]:
# model = Model(input_vocabulary_size=training_set.input_vocabulary_size,
#                   target_vocabulary_size=training_set.target_vocabulary_size,
#                   num_cnn_channels=training_set.image_channels,
#                   input_padding_idx=training_set.input_vocabulary.pad_idx,
#                   target_pad_idx=training_set.target_vocabulary.pad_idx,
#                   target_eos_idx=training_set.target_vocabulary.eos_idx,
#                   embedding_dimension=25, encoder_hidden_size=100, num_encoder_layers=1,
#                   encoder_dropout_p=0.3, encoder_bidirectional=True, num_decoder_layers=1,
#                   decoder_dropout_p=0.3, decoder_hidden_size=100, cnn_kernel_size=7, cnn_dropout_p=0.1,
#                   cnn_hidden_num_channels=50, output_directory="models", conditional_attention=True, auxiliary_task=False,
#                   simple_situation_representation=True, attention_type="bahdanau")
model = Model(input_vocabulary_size=training_set.input_vocabulary_size,
                  target_vocabulary_size=training_set.target_vocabulary_size,
                  num_cnn_channels=training_set.image_channels,
                  input_padding_idx=training_set.input_vocabulary.pad_idx,
                  target_pad_idx=training_set.target_vocabulary.pad_idx,
                  target_eos_idx=training_set.target_vocabulary.eos_idx,
                  **cfg)
model = model.cuda()

/home/chang/anaconda3/envs/gsCAN/lib/python3.8/site-packages/torch/nn/modules/rnn.py:47: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [7]:
model.train()

Model(
  (situation_encoder): ConvolutionalNet(
    (conv_1): Conv2d(16, 50, kernel_size=(1, 1), stride=(1, 1))
    (conv_2): Conv2d(16, 50, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (conv_3): Conv2d(16, 50, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
    (dropout): Dropout(p=0.1, inplace=False)
    (relu): ReLU()
    (layers): Sequential(
      (0): ReLU()
      (1): Dropout(p=0.1, inplace=False)
    )
  )
  (visual_attention): Attention(
    (key_layer): Linear(in_features=150, out_features=100, bias=False)
    (query_layer): Linear(in_features=100, out_features=100, bias=False)
    (energy_layer): Linear(in_features=100, out_features=1, bias=False)
  )
  (encoder): EncoderRNN(
    EncoderRNN
     bidirectional=True 
     num_layers=1
     hidden_size=100
     dropout=0.3
     n_input_symbols=21
    
    (embedding): Embedding(21, 25, padding_idx=0)
    (dropout): Dropout(p=0.3, inplace=False)
    (lstm): LSTM(25, 100, dropout=0.3, bidirectional=True)
  )
  (enc_h

In [8]:
rollout = Rollout(model, 0.8)

In [50]:
for (input_batch, input_lengths, _, situation_batch, _, target_batch, target_lengths, agent_positions, target_positions) in training_set.get_data_iterator(batch_size=10):
    target_scores, target_position_scores = model(commands_input=input_batch, commands_lengths=input_lengths,
                                                          situations_input=situation_batch, target_batch=target_batch,
                                                          target_lengths=target_lengths)
    samples = model.sample(batch_size=10, length=max(target_lengths).astype(int), commands_input=input_batch, commands_lengths=input_lengths, 
                                    situations_input=situation_batch, target_batch=target_batch, sos_idx=training_set.input_vocabulary.sos_idx, eos_idx=training_set.input_vocabulary.eos_idx)
    reward = rollout.get_reward(target_scores, 16, input_batch, input_lengths, situation_batch, target_batch, training_set.input_vocabulary.sos_idx, training_set.input_vocabulary.eos_idx, reward_func)
    prob = model.pred(commands_input=input_batch, commands_lengths=input_lengths,
                                                  situations_input=situation_batch, target_batch=samples,
                                                  target_lengths=target_lengths)
    break

/home/chang/multimodal_neural_gsCAN/seq2seq/model.py:308: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  token = F.softmax(output).multinomial(1).squeeze()


In [11]:
target_scores.shape

torch.Size([10, 11, 9])

In [32]:
samples.shape

torch.Size([110])

In [30]:
F.log_softmax(target_scores, dim=-1).max(dim=-1)[1].detach().chunk(11, dim=1)[1:].szie()

AttributeError: 'tuple' object has no attribute 'szie'

In [11]:
np.array(model.sample(F.log_softmax(target_scores, dim=-1).max(dim=-1)[1].detach()[:,:6], 
             commands_input=input_batch,
             commands_lengths=input_lengths,
             situations_input=situation_batch, 
             target_batch=target_batch,
             sos_idx=testing_set.target_vocabulary.sos_idx, eos_idx=testing_set.target_vocabulary.eos_idx)).T

array([[1, 0, 4, 8, 8, 8, 8, 5, 8, 2, 0, 8, 8, 2, 8, 2, 1, 0, 8, 3, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 2, 8, 8, 8, 8, 8, 3, 5, 3, 4, 2, 8, 4, 8, 2, 8, 4, 8, 2, 6,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 5, 8, 8, 2, 8, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 8, 3, 8, 8, 8, 2, 2, 8, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 4, 5, 8, 8, 8, 8, 3, 8, 8, 3, 4, 5, 3, 2, 6, 2, 8, 1, 0, 8,
        2, 1, 0, 8, 2, 1, 0, 0, 0, 0],
       [1, 0, 5, 8, 3, 8, 8, 8, 3, 5, 3, 4, 8, 2, 8, 2, 8, 2, 1, 0, 8, 2,
        6, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 4, 2, 8, 8, 3, 8, 4, 8, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 5, 8, 8, 2, 8, 2, 8, 2, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 5, 8, 4, 8, 8, 5, 8, 2, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0,

In [51]:
reward

array([0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2])

In [21]:
F.log_softmax(target_scores, dim=-1).max(dim=-1)

RuntimeError: CUDA error: device-side assert triggered

In [19]:
def reward_func(pred):
    return 

In [11]:
rollout.get_reward(target_scores, 16, input_batch, input_lengths, situation_batch, target_batch, testing_set.target_vocabulary.sos_idx, testing_set.target_vocabulary.eos_idx, reward_func)

array([0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2])

In [24]:
target_batch

tensor([[1, 3, 4, 5, 2, 0, 0, 0, 0, 0, 0],
        [1, 3, 4, 5, 4, 5, 2, 0, 0, 0, 0],
        [1, 3, 4, 5, 4, 5, 2, 0, 0, 0, 0],
        [1, 3, 4, 5, 4, 5, 2, 0, 0, 0, 0],
        [1, 3, 4, 5, 4, 5, 4, 5, 2, 0, 0],
        [1, 3, 4, 5, 4, 5, 4, 5, 2, 0, 0],
        [1, 3, 4, 5, 4, 5, 4, 5, 2, 0, 0],
        [1, 3, 4, 5, 4, 5, 4, 5, 2, 0, 0],
        [1, 3, 4, 5, 4, 5, 4, 5, 4, 5, 2],
        [1, 3, 4, 5, 4, 5, 4, 5, 4, 5, 2]], device='cuda:0')